In [2]:
import os
import time
import sys
import warnings

sys.path.append("/home/prakank/anaconda3/lib/python3.8/site-packages/")

import scipy
import numpy as np
from cvxopt import matrix, solvers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from python.svmutil import *

In [3]:
BINARY_CLASSIFICATION = True
LAST_DIGIT = 1
BASE_DIR = "../"
train_path = os.path.join(BASE_DIR, "data", "mnist","train.csv")
test_path  = os.path.join(BASE_DIR, "data", "mnist","test.csv")

In [4]:
def load_data(filename, Binary):
    data = np.genfromtxt(filename,delimiter=',')
    data_x = data[:,:784]/255
    data_y = data[:,784]
    data_y = data_y.reshape((data_y.shape[0],1))
    
    if Binary:
        data_x = data_x[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = data_y[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = -1.0*(data_y==LAST_DIGIT) + 1.0*(data_y==((LAST_DIGIT+1)%10))
        
    return data_x,data_y

In [5]:
def linear_kernel(X,y):
    mat = np.array(X*y)
    return np.dot(mat, mat.T)

def gaussian_kernel_element(X1,X2,gamma):
    return np.exp(-(np.linalg.norm(X1-X2)**2) * gamma)

def gaussian_kernel(X,gamma):
    X_sq   = np.sum(np.multiply(X, X),axis=1, keepdims=True)
    kernel_partial = X_sq + X_sq.T
    kernel_partial = kernel_partial - 2*np.dot(X,X.T)
    kernel = np.power(np.exp(-gamma),kernel_partial)
    return kernel

    kernel = np.zeros((X.shape[0],X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            kernel[i,j] = gaussian_kernel_element(X[i],X[j],gamma)
    return Kernel

In [26]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [6]:
class SVM:
    def __init__(self,kernel,C,threshold=1e-5,gamma=0.05):
        if kernel == "linear":
            self.kernel = linear_kernel
        else:
            self.kernel = gaussian_kernel
            
        self.C = float(C)
        self.threshold = threshold
        self.gamma = gamma
    
    def train(self, X_train, Y_train):
        # minimizing function
        P = 0
        if self.kernel == linear_kernel:
            P = self.kernel(X_train,Y_train)
        elif self.kernel == gaussian_kernel:
            kernel = matrix(self.kernel(X_train, self.gamma))
            P = (kernel*Y_train)*(Y_train.T)

        P = matrix(.5 * (P + P.T))  # Just to be on the safe side (ensuring P is symmetric)
        q = matrix(-1.0*np.ones((X_train.shape[0],1)))
        c = 0.0
        
        # Inequalities
        pos = np.diag(np.ones(X_train.shape[0]))
        neg = np.diag(-np.ones(X_train.shape[0]))
        G   = matrix( np.vstack((neg,pos)) )
        
        zer   = np.zeros((X_train.shape[0]))
        c_val = self.C*np.ones(X_train.shape[0])
        h     = matrix(np.concatenate((zer,c_val)))
        
        # Equality
        A = matrix(Y_train.reshape((1,Y_train.shape[0])))
        b = matrix(0.0)
        
        solvers.options['show_progress'] = True
        sol = solvers.qp(P, q, G, h, A, b);
        
        alpha = np.array(sol['x'])
        self.support_vector_flag = (alpha > self.threshold).reshape(-1)
        # self.support_vector_indices = (np.arange(len(alpha)))[self.support_vector_flag]
        self.alpha = alpha[self.support_vector_flag]
        self.support_vector_x = X_train[self.support_vector_flag]
        self.support_vector_y = Y_train[self.support_vector_flag]
        
        # return self.support_vector_x, self.support_vector_y, self.alpha
        
        if self.kernel == linear_kernel:
            w_partial = self.support_vector_x * self.support_vector_y
            self.w = np.sum(w_partial * self.alpha, axis=0)
            
            b1 = np.min(X_train[(Y_train == 1).reshape(-1)] * self.w)
            b2 = np.max(X_train[(Y_train ==-1).reshape(-1)] * self.w)
            self.b  = (b1+b2)*(-0.5)
        else:
            self.w = None
            
            b1 = float("inf")
            b2 = -float("inf")
            
            for i in range(len(self.alpha)):
                val = 0
                for j in range(len(self.alpha)):
                    val += self.alpha[j] * self.support_vector_y[j] * gaussian_kernel_element(self.support_vector_x[j], self.support_vector_x[i], gamma=self.gamma)
                if self.support_vector_y[i] == 1:
                    b1 = min(b1, val)
                else:
                    b2 = max(b2,val)
            if b1 == float("inf"):
                b1 = 0
            if b2 == -float("inf"):
                b2 = 0                
            self.b = (b1+b2)*(-0.5)
            
        return self.alpha, self.w, self.b

    def predict(self, X_test):   
        if self.kernel == linear_kernel:
            Y_pred = (np.dot(X_test,self.w)) + self.b
        else:
            Y_pred = np.zeros((X_test.shape[0]))
            alpha  = self.alpha.reshape(-1)
            support_vector_y = self.support_vector_y.reshape(-1)
            
            for i in range(X_test.shape[0]):
                mat = np.array(list(map(lambda x: gaussian_kernel_element(x,X_test[i],gamma=self.gamma),self.support_vector_x)))
                Y_pred[i] = np.sum(mat*alpha*support_vector_y) + self.b
                # val = 0
                # for index in range(len(self.alpha)):
                #     val += self.alpha[index] \
                #             *self.support_vector_y[index] \
                #             *gaussian_kernel_element(self.support_vector_x[index],X_test[i],gamma=self.gamma)
                # val += self.b
                # Y_pred[i] = val
                
        Y_pred = Y_pred.reshape(-1)
        Y_pred = np.array(list(map(lambda x: -1 if x<0 else 1,Y_pred)))
        return Y_pred

In [7]:
# Part A
if BINARY_CLASSIFICATION:
    X_train, Y_train = load_data(train_path, BINARY_CLASSIFICATION)
    X_test, Y_test  = load_data(test_path, BINARY_CLASSIFICATION)        

In [8]:
# Part A -> (i)
svm_lin = SVM(kernel="linear",C=1)
alpha,w,b = svm_lin.train(X_train,Y_train)

print("Support Vector: {}\nList: {}".format(len(alpha), alpha))
print("W:",w,"\n")
print("b:",b)

     pcost       dcost       gap    pres   dres
 0: -4.1525e+02 -8.7651e+03  5e+04  3e+00  8e-13
 1: -2.4967e+02 -5.2583e+03  1e+04  5e-01  6e-13
 2: -1.3844e+02 -2.0226e+03  3e+03  1e-01  4e-13
 3: -8.5841e+01 -9.5140e+02  2e+03  6e-02  3e-13
 4: -5.2441e+01 -6.5583e+02  1e+03  3e-02  2e-13
 5: -2.7968e+01 -4.4408e+02  7e+02  2e-02  1e-13
 6: -1.2542e+01 -2.5777e+02  4e+02  7e-03  1e-13
 7: -6.8992e+00 -1.6402e+02  2e+02  3e-03  1e-13
 8: -5.1204e+00 -7.1678e+01  8e+01  6e-04  1e-13
 9: -9.2479e+00 -4.0131e+01  3e+01  5e-05  1e-13
10: -1.2240e+01 -3.1494e+01  2e+01  1e-15  1e-13
11: -1.5470e+01 -2.3412e+01  8e+00  1e-15  1e-13
12: -1.6847e+01 -2.0347e+01  4e+00  9e-16  1e-13
13: -1.7813e+01 -1.8851e+01  1e+00  2e-16  1e-13
14: -1.8234e+01 -1.8322e+01  9e-02  2e-15  1e-13
15: -1.8273e+01 -1.8275e+01  2e-03  3e-15  1e-13
16: -1.8274e+01 -1.8274e+01  3e-05  2e-16  1e-13
17: -1.8274e+01 -1.8274e+01  3e-07  7e-15  1e-13
Optimal solution found.
Support Vector: 159
List: [[1.05617414e-01]
 [

In [9]:
Y_validation = svm_lin.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_lin.predict(X_test)
print("Test Set Accuracy (Linear Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

Validation Accuracy: 97.975%
Test Set Accuracy (Linear Kernel): 98.477%


In [10]:
# Part A -> (ii)
svm_gau = SVM(kernel="gaussian",C=1)
alpha,w,b = svm_gau.train(X_train,Y_train)

print("Support Vector: {}\nList: {}".format(len(alpha), alpha))
print("W:",w,"\n")
print("b:",b)

     pcost       dcost       gap    pres   dres
 0: -7.9991e+01 -5.6970e+03  3e+04  2e+00  2e-15
 1: -4.0391e+01 -2.4332e+03  3e+03  1e-01  2e-15
 2: -4.7583e+01 -4.1104e+02  4e+02  1e-02  3e-15
 3: -6.9765e+01 -1.9362e+02  1e+02  3e-03  2e-15
 4: -7.8547e+01 -1.2896e+02  5e+01  9e-04  1e-15
 5: -8.1985e+01 -1.1136e+02  3e+01  1e-14  1e-15
 6: -8.5302e+01 -9.5867e+01  1e+01  2e-14  1e-15
 7: -8.6831e+01 -9.0583e+01  4e+00  2e-14  1e-15
 8: -8.7528e+01 -8.8687e+01  1e+00  9e-16  1e-15
 9: -8.7815e+01 -8.8007e+01  2e-01  2e-15  1e-15
10: -8.7877e+01 -8.7893e+01  2e-02  8e-15  1e-15
11: -8.7883e+01 -8.7883e+01  4e-04  1e-14  1e-15
12: -8.7883e+01 -8.7883e+01  7e-06  4e-15  1e-15
Optimal solution found.
Support Vector: 867
List: [[1.16956991e-02]
 [1.47274402e-04]
 [9.99999994e-01]
 [6.90274826e-02]
 [9.44218471e-02]
 [8.99039114e-01]
 [3.51481788e-03]
 [4.27276253e-02]
 [6.03387922e-01]
 [1.27433346e-02]
 [7.57699927e-02]
 [5.76902550e-02]
 [2.09097995e-02]
 [4.05430717e-02]
 [2.93609754e

In [11]:
Y_validation = svm_gau.predict(X_train)
print("Validation Accuracy: {}%".format(round(100*accuracy_score(Y_validation,Y_train),3)))

Y_pred = svm_gau.predict(X_test)
print("Test Set Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(Y_pred,Y_test),3)))

Validation Accuracy: 100.0%
Test Set Accuracy (Gaussian Kernel): 99.723%


In [28]:
# Part a -> (iii)
model1 = svm_train(Y_train.reshape(-1), X_train, '-s 0 -c 1 -t 0')
label_predict, accuracy, decision_values=svm_predict(Y_test.reshape(-1),X_test,model1, '-q');
print("{}Linear Score: {}% {}".format(color.BOLD,round(100*accuracy_score(label_predict,Y_test),3),color.END ) )

model2 = svm_train(Y_train.reshape(-1), X_train, '-s 0 -c 1 -t 2 -g 0.05')
label_predict_g, accuracy_g, decision_values_g=svm_predict(Y_test.reshape(-1),X_test,model2,'-q');
print("{}RBF Score: {}% {}".format(color.BOLD,round(100*accuracy_score(label_predict_g,Y_test),3),color.END ) )

..*..*
optimization finished, #iter = 4258
nu = 0.008999
obj = -18.274204, rho = 1.219513
nSV = 158, nBSV = 2
Total nSV = 158
Linear Score: 99.031% 
*.*
optimization finished, #iter = 1288
nu = 0.040537
obj = -87.883228, rho = -0.890975
nSV = 848, nBSV = 40
Total nSV = 848
RBF Score: 99.585% 


In [68]:
# Part B
BINARY_CLASSIFICATION = False

if not BINARY_CLASSIFICATION:
    X_train, Y_train = load_data(train_path, BINARY_CLASSIFICATION)
    X_test, Y_test  = load_data(test_path, BINARY_CLASSIFICATION)

In [69]:
def svm_load_data(X,Y,d1,d2):
    X_data = X[(Y == d1).reshape(-1) | (Y == d2).reshape(-1)]
    Y_data = Y[(Y == d1).reshape(-1) | (Y == d2).reshape(-1)]
    Y_data = -1.0*(Y_data == d1) + 1.0*(Y_data == d2)
    return X_data, Y_data

In [72]:
# Part B -> (i)
def one_vs_one_classifier(X_train, Y_train, X_test, Y_test):
    svm_models = {}
    max_val = 1
    for i in range(max_val):
        for j in range(i+1,max(2,max_val)):
            print("Training Phase: {" + str(i) + ", " + str(j) + "}")
            separator = "_"
            index = str(i) + separator + str(j)
            svm_models[index] = {}
            svm_models[index]["X_train_" + index], svm_models[index]["Y_train_" + index] = svm_load_data(X_train,Y_train,i,j)
            svm_models[index]["svm_gaussian"] = SVM(kernel="gaussian",C=1)
            svm_models[index]["svm_gaussian"].train(svm_models[index]["X_train_" + index], svm_models[index]["Y_train_" + index])

    y_pred_train = np.zeros((X_train.shape[0], 10, 10))
    for i in range(max_val):
        for j in range(i+1,max(2,max_val)):
            print("Prediction Phase (Validation): {" + str(i) + ", " + str(j) + "}")
            y_pred = svm_models[index]["svm_gaussian"].predict(X_train)
            temp1  = (y_pred-1)/(-2)
            temp2  = (y_pred+1)/(2)
            y_pred_train[:,i,j] = temp1
            y_pred_train[:,j,i] = temp2
    y_pred_train = np.sum(y_pred_train,axis=2)
    y_pred_train = y_pred_train[:,::-1]
    y_pred_train = np.argmax(y_pred_train,axis=1)
    
    
    y_pred_test = np.zeros((X_test.shape[0], 10, 10))
    for i in range(max_val):
        for j in range(i+1,max(2,max_val)):
            print("Prediction Phase (Test set): {" + str(i) + ", " + str(j) + "}")
            y_pred = svm_models[index]["svm_gaussian"].predict(X_test)
            temp1  = (y_pred-1)/(-2)
            temp2  = (y_pred+1)/(2)
            y_pred_test[:,i,j] = temp1
            y_pred_test[:,j,i] = temp2            
    y_pred_test = np.sum(y_pred_test,axis=2)
    y_pred_test = y_pred_test[:,::-1]
    y_pred_test = np.argmax(y_pred_test,axis=1)
    
    print("Validation Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(y_pred_train, Y_train), 3)))
    print("Test Accuracy (Gaussian Kernel): {}%".format(round(100*accuracy_score(y_pred_test, Y_test), 3)))


In [73]:
one_vs_one_classifier(X_train, Y_train, X_test, Y_test)

{0, 1}

     pcost       dcost       gap    pres   dres
 0: -4.6292e+01 -4.9711e+03  2e+04  2e+00  2e-15
 1: -1.6545e+01 -1.9199e+03  3e+03  1e-01  1e-15
 2: -2.2858e+01 -3.1871e+02  4e+02  1e-02  2e-15
 3: -4.0582e+01 -1.2638e+02  9e+01  3e-03  2e-15
 4: -4.7771e+01 -7.5321e+01  3e+01  6e-04  1e-15
 5: -5.0430e+01 -6.2567e+01  1e+01  2e-04  1e-15
 6: -5.1788e+01 -5.6885e+01  5e+00  4e-05  9e-16
 7: -5.2467e+01 -5.4536e+01  2e+00  1e-14  9e-16
 8: -5.2851e+01 -5.3479e+01  6e-01  2e-15  9e-16
 9: -5.2996e+01 -5.3145e+01  1e-01  4e-15  9e-16
10: -5.3041e+01 -5.3066e+01  2e-02  3e-15  9e-16
11: -5.3050e+01 -5.3051e+01  1e-03  5e-15  1e-15
12: -5.3050e+01 -5.3050e+01  2e-05  1e-14  1e-15
Optimal solution found.


In [43]:
a = np.arange(30).reshape(2, 3, 5)

In [44]:
print(a)

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]]

 [[15 16 17 18 19]
  [20 21 22 23 24]
  [25 26 27 28 29]]]


In [61]:
b = np.sum(a,axis=2)
b[0][1] = 120
print(b.shape)
b = b[:,::-1]
print(b,b.shape)
print(2-np.argmax(b,axis=1))

(2, 3)
[[ 60 120  10]
 [135 110  85]] (2, 3)
[1 2]
